In [ ]:
from segmentandoDatasets import segmentando_datasets
segmentando_datasets(1000,1000,1000)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from tensorflow.keras.optimizers import Adam
import keras 

import os
os.environ['XLA_FLAGS'] = '--xla_gpu_strict_conv_algorithm_picker=false'



In [1]:
from preprocessamento import *

csv_file = 'Datasets_csv/df_PUC.csv'
treino_gerador, validacao_gerador, teste_gerador, x_treino, y_treino, x_teste, y_teste, x_validacao, y_validacao = preprocessamento(csv_file, 0.6, 0.2, 0.2)

2024-09-11 13:46:59.585552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 13:46:59.599321: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 13:46:59.603348: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 13:46:59.612993: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-11 13:47:00.436444: W tensorflow/compiler/tf2

Found 600 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.
Found 200 validated image filenames belonging to 2 classes.


In [ ]:
print(x_treino.shape)

In [5]:
class Autoencoder(tf.keras.Model):
    def __init__(self, input_shape=(256, 256, 3)):
        super(Autoencoder, self).__init__()
        self.input_shape = input_shape
        
        # Definir o encoder
        self.encoder = self.build_encoder()
        # Definir o decoder
        self.decoder = self.build_decoder()

    def build_encoder(self):
        input_img = Input(shape=self.input_shape)
        
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = MaxPooling2D((2, 2), padding='same')(x)
        
        return Model(inputs=input_img, outputs=x, name='encoder')

    def build_decoder(self):
        input_encoded = Input(shape=self.encoder.output_shape[1:])
        
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
        
        return Model(inputs=input_encoded, outputs=decoded, name='decoder')

    def call(self, inputs):
        encoded = self.encoder(inputs)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

In [ ]:
autoencoder = Autoencoder()
autoencoder.build(input_shape=(256, 256, 3)) 
autoencoder.summary()

In [7]:

checkpoint_path = 'weights_parciais/weights-improvement-{epoch:02d}-{loss:.2f}.weights.h5'
cp_callback = ModelCheckpoint(filepath=checkpoint_path, 
                                  save_weights_only=True, 
                                  monitor='loss',
                                  mode='max', 
                                  save_best_only=True, 
                                  verbose=1)

In [ ]:
autoencoder.compile(optimizer='Adam', loss='mean_squared_error')

history = autoencoder.fit(x_treino, x_treino,
                epochs=10,
                batch_size=32,
                shuffle=True,
                callbacks=[cp_callback],
                validation_data=(x_validacao, x_validacao))

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
autoencoder.save("Modelos_keras/Autoencoder.keras.h5")
autoencoder.save_weights("weights_finais/Autoencoder.weights.h5")

In [11]:
autoencoder.load_weights("weights_finais/Autoencoder.weights.h5")

In [ ]:
encoded_imgs = autoencoder.encode(x_teste)
reconstructed_images = autoencoder.decode(encoded_imgs)

In [ ]:
print("Shape das imagens de teste:", x_teste.shape)
print("Shape das imagens codificadas:", encoded_imgs.shape)
print("Shape das imagens reconstruídas:", reconstructed_images.shape)


In [ ]:
print("Forma das imagens codificadas:", encoded_imgs.shape)
print("Forma das labels de treinamento:", y_treino.shape)


In [24]:
encoded_imgs = tf.reshape(encoded_imgs, (encoded_imgs.shape[0], -1))  # Achatar para (batch_size, 32*32*32)


In [25]:
import matplotlib.pyplot as plt
import numpy as np

def plot_original_and_reconstructed(model, X_val, num_images=5):
    """
    Plota as imagens originais e suas versões reconstruídas pelo autoencoder.
    
    Parâmetros:
        model (keras.Model): O modelo do autoencoder treinado.
        X_val (numpy.ndarray): Conjunto de dados de validação.
        num_images (int): Número de imagens a serem plotadas.
    """
    plt.figure(figsize=(12, 6))

    for i in range(num_images):
        image_index = np.random.randint(0, len(X_val))
        original_image = X_val[image_index]

        reconstructed_image = model.predict(original_image.reshape(1, 256, 256, 3))[0]

        plt.subplot(2, num_images, i + 1)
        plt.imshow(original_image)
        plt.title("Original")
        plt.axis("off")

        plt.subplot(2, num_images, i + 1 + num_images)
        plt.imshow(reconstructed_image)
        plt.title("Reconstruída")
        plt.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
plot_original_and_reconstructed(autoencoder, x_teste)

In [ ]:
from preprocessamento import preprocessamento_dataframe_unico

UFPR04, _ = preprocessamento_dataframe_unico("Datasets_csv/df_UFPR04.csv", True)
UFPR05, _ = preprocessamento_dataframe_unico("Datasets_csv/df_UFPR05.csv", True)

In [28]:
X_UFPR04,_ = next(UFPR04)
X_UFPR05,_ = next(UFPR05)

In [ ]:
plot_original_and_reconstructed(autoencoder, X_UFPR04)
plot_original_and_reconstructed(autoencoder, X_UFPR05)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

# Assumindo que você já tem as imagens codificadas e os rótulos
# encoded_imgs shape: (num_amostras, 32, 32, 32)
# labels: lista ou array com rótulos 'empty' ou 'occupied'

# Converter rótulos para formato numérico
label_map = {'empty': 0, 'occupied': 1}
numeric_labels = np.array([label_map[label] for label in labels])

# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(encoded_imgs, numeric_labels, test_size=0.2, random_state=42)

# Criar o modelo
model = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(32, 32, 32)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# Compilar o modelo
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Treinar o modelo
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

# Avaliar o modelo
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nAcurácia no conjunto de teste: {test_acc}')

# Fazer previsões
predictions = model.predict(X_test)
predicted_labels = (predictions > 0.5).astype(int)

# Você pode usar estas previsões para análise adicional ou visualização